In [ ]:
# Setup the environment
!pip install -q --upgrade huggingface_hub
from huggingface_hub import login
# 登录到 Hugging Face Hub
login(token="Hugging_token", add_to_git_credential=True)
!pip install -q transformers==4.38.0 accelerate==0.27.2 bitsandbytes==0.42.0 peft datasets==2.17.1 trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 8.0 MB/s eta 0:00:00
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB

In [ ]:
!nvidia-smi

Tue Mar 12 03:41:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from peft import LoraConfig
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, \
                        get_cosine_schedule_with_warmup, set_seed
from peft import LoraConfig, TaskType, get_peft_model
from accelerate import Accelerator, notebook_launcher
from accelerate.utils import set_seed
from accelerate.logging import get_logger
import logging
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.notebook import tqdm
import torch
from torch.nn.utils.rnn import pad_sequence
import glob
from collections import OrderedDict
import re
import os
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForSequenceClassification
os.environ["TOKENIZERS_PARALLELISM"] = "false"
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    bnb_4bit_use_double_quant=True,
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# raw_datasets = load_dataset("csv", data_files="/kaggle/input/quotes/quotes.csv")
import re
import json
import pandas as pd
from datasets import load_dataset
# dataset = load_dataset("OpenAssistant/oasst_top1_2023-08-25")
dataset = load_dataset('csv',data_files="/content/output_2048.csv")

raw_datasets = dataset.map(lambda samples: tokenizer(samples["text"],max_length=2048,truncation=True), batched=True)
len(raw_datasets['train'])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

12

In [ ]:
raw_datasets['train']

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 12
})

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.4 MB/s eta 0:00:00


In [ ]:
import transformers
from trl import SFTTrainer
import wandb

# Set your WandB API key
wandb.login(key="WandB_key")

def formatting_func(example):
    return example['text']

trainer = SFTTrainer(
    model=model,
    train_dataset=raw_datasets['train'],
    max_seq_length=2048,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=2,
        warmup_steps=2,
        num_train_epochs=15,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)
trainer.train()
# 保存模型
trainer.save_model('./outputs')


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
wandb: Currently logged in as: xiaoqidaozhang. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,3.917100
2,4.268200
3,4.557700
4,3.634200
5,4.240800
6,3.944500
7,3.783000
8,3.557900
9,3.155700
10,3.562500


In [ ]:
import os
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
LORA_WEIGHTS = "./outputs/"
model.eval()
model = PeftModel.from_pretrained(model, LORA_WEIGHTS)
print(model)
model = model.to("cuda")

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_fea

In [ ]:
prompt = "<|im_start|>user 查看市区交通拥堵指数<|im_end|>"
inp = tokenizer(prompt, max_length=512, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids=inp["input_ids"], max_new_tokens=1024)
print(tokenizer.decode(outputs[0]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<bos><|im_start|>user 查看市区交通拥堵指数<|im_end|> <|im_start|>assistant ---

**当前市区交通拥堵指数：78.23%**

**拥堵程度：**严重

**原因：**

* **交通流量过大：**市区交通流量目前已超过平均流量的 120%，导致交通拥堵现象加剧。
* **交通工具使用率高：**使用地铁、公交车等公共交通的比例已超过 50%，使用私车比例却已降低至 20%以下。
* **天气因素：**今天天气温热，风力强，这些因素加剧了交通拥堵的风险。

**建议：**

* **减少非必要出行：**利用公共交通、骑自行车或步行出行，减少对市区交通的依赖。
* **提高交通效率：**优化交通信号、改善道路设施，提高交通流速。
* **缓解交通拥堵：**利用交通科技手段，预见和应对拥堵情况，减少拥堵带来的负面影响。

**信息来源：**

* **市政府交通部门**

**注意：**

* 以上信息仅供参考，具体情况请以实际情况进行查询。
* 拥堵指数是相对指标，不同方法和标准可能导致指标值差异较大。
* 拥堵指数超过 100% 表示交通拥堵现象严重，可能导致交通事故发生增加。

<|im_end|> <|im_end|>assistant ---

**其他信息：**

* **当前天气：**高温36°C，风力8级
* **交通工具使用率：**使用地铁、公交车等公共交通比例超过50%，使用私车比例仅为20%
* **交通流量：**超过平均流量的120%，严重拥堵
* **拥堵程度：**严重，交通拥堵现象加剧

**分析：**

市区交通拥堵情况严重，当前拥堵指数已超过 100%。主要原因是交通流量过大、交通工具使用率高、天气因素加剧了交通拥堵的风险。

**建议：**

* 减少非必要出行，利用公共交通、骑自行车或步行出行，减少对市区交通的依赖。
* 提高交通效率，优化交通信号、改善道路设施，提高交通流速。
* 利用交通科技手段，预见和应对拥堵情况，减少拥堵带来的负面影响。<eos>


In [ ]:
test_prompt = '''
<|im_start|>user
通过内容完善名为方源的角色刻画
你可以根据角色的实际情况，填写每个部分的相关信息，甚至对信息进行补充。这个模板可帮助你系统性地构建一个深度刻画的角色形象。


第三节：请一边玩蛋去
声音传入高脚吊楼，方源睁开干涩的眼皮，心中暗道：“是五更天了。”
“方源少爷，您醒了。奴家这就上楼来，伺候您洗漱。”就在此刻，楼下传来一个少女的声音。
“若是哪天，沈翠能伺候我这样，该是什么滋味？”方正心中有些想，又有些不敢想。
“好了，不用收拾了。”方源不耐地拂开沈翠的柔软小手，衣衫早就平整，沈翠更多的是在引诱。
“你退下罢。”方源看也不看沈翠，整理着自己的袖口。
沈翠微微撅嘴，为方源今日的不解风情感到有些奇怪和委屈。想要说什么撒娇的话，但是被方源若有若无的莫名气质震慑着，张口几次，最终说了声“是”，乖乖地退下。…,
“你准备好了？”方源看向方正。
但此时点破毫无意义，淡淡地吩咐着：“那就走吧。”
“你们看，那是方家两兄弟。”小耳边传来小心翼翼的议论声。
“前面走着的就是那个方源，就是那个作诗的方源。”有人强调着。
“原来是他呀，面无表情、旁若无人的样子，果真和传闻中一样拽。”有人语气酸酸，带着嫉妒和羡慕。
“哼，你要是能像他一样，你也可以这样拽！”有人冷哼着这样回答，隐藏着一种不满。
他感到胸口传来一阵阵的压抑，甚至是呼吸不畅，这该死的感觉让他甚至联想到“窒息”这个词！
“哼，这样的议论，果真是木秀于林风必催之。”听着耳边的议论声，方源心中冷笑着。
“但是，我却不想这么做呀……”方源在心中悠然一叹。<|im_end|>
'''

In [ ]:
inp = tokenizer(test_prompt, max_length=2048, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids=inp["input_ids"], max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

<bos>
<|im_start|>user 
通过内容完善名为方源的角色刻画
你可以根据角色的实际情况，填写每个部分的相关信息，甚至对信息进行补充。这个模板可帮助你系统性地构建一个深度刻画的角色形象。


第三节：请一边玩蛋去
声音传入高脚吊楼，方源睁开干涩的眼皮，心中暗道：“是五更天了。”
“方源少爷，您醒了。奴家这就上楼来，伺候您洗漱。”就在此刻，楼下传来一个少女的声音。
“若是哪天，沈翠能伺候我这样，该是什么滋味？”方正心中有些想，又有些不敢想。
“好了，不用收拾了。”方源不耐地拂开沈翠的柔软小手，衣衫早就平整，沈翠更多的是在引诱。
“你退下罢。”方源看也不看沈翠，整理着自己的袖口。
沈翠微微撅嘴，为方源今日的不解风情感到有些奇怪和委屈。想要说什么撒娇的话，但是被方源若有若无的莫名气质震慑着，张口几次，最终说了声“是”，乖乖地退下。…,
“你准备好了？”方源看向方正。
但此时点破毫无意义，淡淡地吩咐着：“那就走吧。”
“你们看，那是方家两兄弟。”小耳边传来小心翼翼的议论声。
“前面走着的就是那个方源，就是那个作诗的方源。”有人强调着。
“原来是他呀，面无表情、旁若无人的样子，果真和传闻中一样拽。”有人语气酸酸，带着嫉妒和羡慕。
“哼，你要是能像他一样，你也可以这样拽！”有人冷哼着这样回答，隐藏着一种不满。
他感到胸口传来一阵阵的压抑，甚至是呼吸不畅，这该死的感觉让他甚至联想到“窒息”这个词！
“哼，这样的议论，果真是木秀于林风必催之。”听着耳边的议论声，方源心中冷笑着。
“但是，我却不想这么做呀……”方源在心中悠然一叹。<|im_end|>
**角色刻画模板**

**1. 基本信息：**
- **姓名：** 方源
- **年龄：** 不确定（猜测：可能是中年到老年之间）
- **性别：** 男性
- **外貌特征：**
  - 身高：不确定
  - 发型：不确定
  - 眼睛颜色：不确定
  - 其他特征：面无表情，旁若无人的样子，冷漠气质

**2. 个性特征：**
- **性格描述：** 冷漠，有些不解风情，有时霸道
- **兴趣爱好：** 对修行和力量的追求，对自身形象有所在意

**3. 言谈模式：**
- **词汇选择：** 沉稳，有控制
- **语法结构：** 直接，有时霸道
- **语速和语调：** 冷静，有控制

**4. 行为举止：**
